In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d thedevastator/cancer-patients-and-air-pollution-a-new-link
!unzip \*.zip && rm *.zip

  0% 0.00/7.36k [00:00<?, ?B/s]
100% 7.36k/7.36k [00:00<00:00, 14.6MB/s]
Archive:  cancer-patients-and-air-pollution-a-new-link.zip
  inflating: cancer patient data sets.csv  


In [4]:
!pip install scikit-fuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894078 sha256=110386a5f16ed5fbf07ac12e159bce18795b0d69f6d0b578ee9892484188ef8b
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import skfuzzy as fuzz
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load data
data = pd.read_csv('/content/cancer patient data sets.csv')

# Assuming 'Level' is the name of your target variable
X = data.drop(['index', 'Patient Id', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards', 'Genetic Risk', 'Level'], axis=1)
y = data['Level']

# Encode categorical variables (Gender)
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=32)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert target labels to fuzzy values
fuzzy_y_train = np.zeros((len(y_train), 3))
for i, label in enumerate(y_train):
    if label == 'Low':
        fuzzy_y_train[i] = [1, 0, 0]
    elif label == 'Medium':
        fuzzy_y_train[i] = [0, 1, 0]
    else:  # Assuming 'High'
        fuzzy_y_train[i] = [0, 0, 1]

# Instantiate and train FKNN
fuzzy_k = 2  # You can choose the value of k
fuzzy_membership = fuzz.cmeans(X_train_scaled.T, fuzzy_k, 2, error=0.005, maxiter=1000, init=None)
fuzzy_labels = fuzz.cmeans_predict(X_test_scaled.T, fuzzy_membership[0], 2, error=0.005, maxiter=1000)

# Convert fuzzy labels to crisp labels
y_pred = []
for col in fuzzy_labels[0].T:  # Access the first array in the tuple and then transpose it
    max_index = np.argmax(col)
    if max_index == 0:
        y_pred.append('Low')
    elif max_index == 1:
        y_pred.append('Medium')
    else:
        y_pred.append('High')

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Specify average='weighted'
recall = recall_score(y_test, y_pred, average='weighted')  # Specify average='weighted'
f1 = f1_score(y_test, y_pred, average='weighted')  # Specify average='weighted'

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
# Display classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.36
Precision: 0.20948646325741302
Recall: 0.36
F1 Score: 0.2635907643312102

Classification Report:
               precision    recall  f1-score   support

        High       0.00      0.00      0.00       146
         Low       0.52      0.97      0.68       119
      Medium       0.16      0.21      0.18       135

    accuracy                           0.36       400
   macro avg       0.23      0.39      0.29       400
weighted avg       0.21      0.36      0.26       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13